# Data Inspection
Loads a TWIX scan, extracts k-space, zero-fills it, and displays a preview as an animated GIF. This helps you determine the number of phase-encode lines collected and the offset. Then you can update those values in your configuration file accordingly.

### Loading packages and data

In [1]:
import yaml
import numpy as np
import utils.data_ingestion as di
import utils.gif as gif
from IPython.display import display

def load_config(config_file="config.yaml"):
    """
    Load configuration from a YAML file.

    Parameters
    ----------
    config_file : str
        Path to the YAML configuration file.

    Returns
    -------
    dict
        Parsed configuration data.
    """
    with open(config_file, "r") as f:
        config = yaml.safe_load(f)
    return config

# Read configuration
config = load_config()

# Extract paths and other parameters from config
twix_file = config["data"]["twix_file"]
dicom_folder = config["data"]["dicom_folder"]

# Read TWIX file (only the last scan by default in this example)
scans = di.read_twix_file(twix_file, include_scans=[-1], parse_pmu=False)

Software version: VD/VE (!?)

Scan  1


100%|██████████| 287M/287M [00:00<00:00, 1.19GB/s]

Read 1 scans from 20250307_JM/raw/meas_MID00115_FID27380_Cor_250306_2.dat


### Extracting k-space data, zero-filling, and displaying

In [2]:
# Extract raw k-space data
kspace = di.extract_image_data(scans)

n_frames = di.get_num_frames(dicom_folder)
n_coils = kspace.shape[1]

# Reshape k-space into frames
kspace = np.reshape(kspace, (n_frames, -1, n_coils, kspace.shape[2]))

# Pull the total number of phase encodes and define offset
extended_pe_lines = di.get_total_phase_encodes(dicom_folder)
offset = 32  # Adjust if needed

# Allocate zero-filled array
kspace_zf = np.zeros((n_frames, extended_pe_lines, n_coils, 256), dtype=np.complex64)
kspace_zf[:, offset : offset + kspace.shape[1], :] = kspace

# Display k-space as an animated GIF
preview = gif.display_kspace_as_gif(kspace_zf, duration=0.2)
display(preview)

Extracted image data shape: (4800, 30, 256)
Found 100 DICOM frames in folder 20250307_JM/dcm/SE00028001 COR_250306_2
Total phase encodes from AcquisitionMatrix: 128
